In [152]:
import os 
import pandas as pd
from collections import Counter
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import *
stemmer = PorterStemmer()

In [153]:
def stem(data):
    
    new = {}
    for k,v in data.items():
        new[k] = stemmer.stem(k)
    for old,new in new.items():
        data[new] = data.pop(old) 
        
    return data

In [154]:
def condProb(data,total_spam,total_ham,nof_spam,nof_ham):
    
    n = len(data)
    print(n)
    for word in data.keys():
        #cond_spam  = (data[word][0]+1)/(total_spam-data[word][0] + n -1)
        #cond_ham   = (data[word][1]+1)/(total_ham-data[word][1] + n -1)
        cond_spam = (data[word][0]+1)/(total_spam+ n*1)
        cond_ham = (data[word][1]+1)/(total_ham + n*1)
        data[word] = data[word] + [cond_spam,cond_ham]
    print(nof_spam)
    print(nof_ham)
    prior_spam = nof_spam / (nof_spam+nof_ham)
    prior_ham = nof_ham / (nof_spam+nof_ham)
    print(prior_spam)
    
    return data,prior_spam,prior_ham

In [155]:
def train():
    data = {}
    total_spam = 0
    total_ham = 0
    nof_spam = 0
    nof_ham = 0
    
    for filename in os.listdir(".\ham"):
        file = open('.\ham' + '\\' + filename, errors='ignore')
        wordcount = Counter(file.read().split())
        for item in wordcount.items():
            if item[0] in data:
                data[item[0]][1] += item[1]
            else:
                data[item[0]] = [0,item[1]]
            total_ham += item[1]
        nof_ham += 1

    for filename in os.listdir('.\spam'):
        file = open('.\spam'+'\\' +filename,errors='ignore')
        wordcount = Counter(file.read().split())
        for item in wordcount.items():
            if item[0] in data:
                 data[item[0]][0] += item[1]
            else:
                data[item[0]] = [item[1],0]
            total_spam += item[1]
        nof_spam += 1
    #print(nof_spam)
    data = stem(data)
    data_new,ps,ph = condProb(data,total_spam,total_ham,nof_spam,nof_ham)
    return data_new,ps,ph
    


In [134]:
#d1,prior_spam,prior_ham = train()
#print(d1)

8885
123
340
0.265658747300216
{'subject:': [123, 340, 0.0037296598189310312, 0.0040937357439554374], 're': [20, 137, 0.0006316359370770295, 0.0016567024418353382], ':': [370, 1701, 0.011158901555027521, 0.020432663449302505], 'issu': [0, 2, 3.0077901765572835e-05, 3.601527047468127e-05], 'fyi': [0, 29, 3.0077901765572835e-05, 0.00036015270474681264], '-': [470, 7073, 0.014166691731584805, 0.08492400777929843], 'see': [1, 0, 6.015580353114567e-05, 1.2005090158227088e-05], 'note': [0, 1, 3.0077901765572835e-05, 2.4010180316454177e-05], 'below': [13, 44, 0.00042109062471801966, 0.000540229057120219], 'alreadi': [1, 10, 6.015580353114567e-05, 0.00013205599174049797], 'done': [2, 26, 9.02337052967185e-05, 0.0003241374342721314], '.': [1448, 2969, 0.04358287965831504, 0.03565511776993445], 'stella': [1, 30, 6.015580353114567e-05, 0.00037215779490503976], 'by': [38, 225, 0.0011730381688573406, 0.0027131503757593218], 'l': [14, 103, 0.0004511685264835925, 0.0012485293764556173], 'morri': [1, 

In [ ]:
stop_words = set(stopwords.words('english'))
stop = []
for k in d1.keys():
    if k in stop_words:
        stop.append(k)
for k in stop:
    del d1[k]

#print(d1)

In [ ]:
def condProb(data,total_spam,total_ham,nof_spam,nof_ham):
    
    n = len(data)
    for word in data.keys():
    #cond_spam  = (data[word][0]+1)/(spam_total-data[word][0] + n -1)
    #cond_ham   = (data[word][1]+1)/(ham_total-data[word][1] + n -1)
        cond_spam = (data[word][0]+1)/(total_spam+ n*1)
        cond_ham = (data[word][1]+1)/(total_ham + n*1)
        data[word] = data[word] + [cond_spam,cond_ham]
    
    prior_spam = nof_spam / (nof_spam+nof_ham)
    prior_ham = nof_ham / (nof_spam+nof_ham)
    print(prior_spam)
    
    return data

In [ ]:

updated = condProb(d1)
print(updated)

In [ ]:
d = {'Apples':1,'Oranges':2}
e = {}
for k,v in d.items():
    e[k] = stemmer.stem(k)
    
#print(e)

for k,v in e.items():
    d[v] = d.pop(k)
    
#print(d)
    
updated_new = {}
for k,v in updated.items():
    updated_new[k] = stemmer.stem(k)
#print(updated_new)
    


    
    


In [ ]:
d = {"I":1, "of":2, "as":3, "cool":4, "person":5}
l = []
stop_words = set(stopwords.words('english'))
for k in d.keys():

    if k in stop_words:
    
        l.append(k)
for k in l:
    del d[k]
print(d)

In [166]:
def predict(text,d1,prior_spam,prior_ham):
    file = open(text,errors="ignore")
    wordcount = Counter(file.read().split())
    #print(wordcount.items())
    l = []
    stop_words = set(stopwords.words('english'))
    for k in wordcount:
        #print(k)
        if k in stop_words:
            l.append(k)
    #print(l)
    for k in l:
        del wordcount[k]
        
    #print(wordcount)
    
    spam_score = np.log(prior_spam)
    ham_score = np.log(prior_ham)
    for item in wordcount.items():
        #print(item)
        text = item[0]
        
        count= item[1]
        #print(count)
        if text in d1:
            spam_score += np.log(d1[text][2])*count
            ham_score += np.log(d1[text][3])*count

    if spam_score>=ham_score:
        return "spam"
    else:
        return "ham"

In [157]:
#predict(r"C:\Users\what's in\Desktop\Naive Bayes\homework2\test\spam\0455.2004-02-16.GP.spam.txt")

In [167]:
def accuracy(d1,prior_spam,prior_ham):
    tp = 0
    tn = 0 
    fp = 0
    fn = 0
    path1 = r"C:\Users\what's in\Desktop\Naive Bayes\homework2\test\ham"
    for filename in os.listdir(path1):
        x = predict(path1 + "//" + filename,d1,prior_spam,prior_ham)
        if x== "spam":
            fn += 1
        if x == "ham":
            tp += 1
        
    path2 = r"C:\Users\what's in\Desktop\Naive Bayes\homework2\test\spam"
    for filename in os.listdir(path2):
        x = predict(path2 + "//" + filename,d1,prior_spam,prior_ham)
        if x == "spam":
            tn += 1
        if x == "ham":
            fp += 1

    accuracy = (tp+tn)/(tp+tn+fp+fn)
    return accuracy


In [151]:
#accuracy = (tp+tn)/(tp+tn+fp+fn)
print(accuracy())

0.897489539748954


In [168]:
def main():
    d1,prior_spam,prior_ham = train()
    print(accuracy(d1,prior_spam,prior_ham))
    

In [169]:
if __name__ == "__main__":
    main()

8885
123
340
0.265658747300216
0.897489539748954
